This program summarizes drug reviews from the Drug Review Dataset from UCI Machine Learning Repository.  May need to run on HPC to get results for whole dataset.  

In [1]:
import pandas as pd
import numpy as np
# import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import gensim
from gensim.models import Word2Vec

ModuleNotFoundError: No module named 'gensim'

In [ ]:
reviews = pd.read_table("drugsComTrain_raw.tsv") # Read a tsv into a DataFrame
print(reviews.head(0))                      
print(reviews.shape)
reviews.head()

In [3]:
# Check for any nulls values
reviews.isnull().sum()
# Remove null values 
reviews = reviews.dropna()
reviews = reviews.drop(['Unnamed: 0','condition','rating','date','usefulCount'], 1)
reviews = reviews.reset_index(drop=True)

In [4]:
reviews.head()

,drugName,review
0,Valsartan,"""It has no side effect, I take it in combinati..."
1,Guanfacine,"""My son is halfway through his fourth week of ..."
2,Lybrel,"""I used to take another oral contraceptive, wh..."
3,Ortho Evra,"""This is my first time using any form of birth..."
4,Buprenorphine / naloxone,"""Suboxone has completely turned my life around..."


In [5]:
# Inspecting some of the reviews
for i in range(5):
    print("Review #",i+1)
    print(reviews.review[i])

Review # 1
"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"
Review # 2
"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. 
We have tried many different medications and so far this is the most effective."
Review # 3
"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. Bu

In [6]:
# A list of contractions 
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [7]:
stops = set(stopwords.words("english"))

In [8]:
def clean_review(reviews, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    reviews['review'] = [review.lower() for review in reviews['review']]
    
    # Replace contractions with their longer forms 
    reviews['review'] = [check_contractions(review) for review in reviews['review']]
        
    # Format words and remove unwanted characters
    reviews['review'] = [re.sub(r'https?:\/\/.*[\r\n]*', '', review, flags=re.MULTILINE) for review in reviews['review']]
    reviews['review'] = [re.sub(r'\<a href', ' ', review) for review in reviews['review']]
    reviews['review'] = [re.sub(r'&amp;', '', review) for review in reviews['review']]
    reviews['review'] = [re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', review) for review in reviews['review']]
    reviews['review'] = [re.sub(r'<br />', ' ', review) for review in reviews['review']]
    reviews['review'] = [re.sub(r'\'', ' ', review) for review in reviews['review']]
    
    # Optionally, remove stop words
    if remove_stopwords:
        reviews['review'] = [remove_stopwords(review) for review in reviews['review']]

    return reviews

def check_contractions(review):
    new_review = []
    for word in review.split(" "):
        if word in contractions:
            new_review.append(contractions[word])
        else:
            new_review.append(word)
    return " ".join(new_review)

def remove_stopwords(review):
    review = review.split()
    review = [w for w in reviews['review'] if not w in stops]
    return " ".join(reviews)

In [9]:
clean_review(reviews.loc[:5], False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,drugName,review
0,Valsartan,it has no side effect i take it in combinati...
1,Guanfacine,my son is halfway through his fourth week of ...
2,Lybrel,i used to take another oral contraceptive wh...
3,Ortho Evra,this is my first time using any form of birth...
4,Buprenorphine / naloxone,suboxone has completely turned my life around...
